**Date**: 2018-08-23

**Authors**: Zhanyuan Zhang

**Purpose**: Apply convolutional neural network (CNN) to train the binary classificaton model.
- Use operations like 1D convolution, maxpooling, and dropout to improve accuracy.
- Add bias in layers.
- Use Relu for learning session, and softmax for final classification.

**Background**: Current model trained by recurent neural network (RNN) gives low accuracy. The reason why we switch from CNN to RNN was that this somehow improved the accuracy by 10%. However, given the math behind these two neural networks, CNN should be better in handling spacial data, which is in our case, since the order in a nucleotide sequence does matter.

**Experiment**:

In [2]:
import pickle
import numpy as np
import matplotlib.pyplot as plt
from utility import flatten
from utility import curtail
from utility import prepare_input
from utility import to_np_array
from utility import unpickle

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/matplotlib/__init__.py:1067: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #2
  (fname, cnt))
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/matplotlib/__init__.py:1067: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #3
  (fname, cnt))


In [5]:
real_buffer_path = "/home/ubuntu/data2/10_percent/random_0.1_instance_7.txt"
# random_buffer_path = "/home/ubuntu/formatted/random_sequences/random_sequence_buffer.txt"
curtail_len = 3000
motif_num = 3

In [6]:
seq_record_list = unpickle(real_buffer_path)
len(seq_record_list)

8088

In [7]:
import random
from random import shuffle

first_list = [] # to add to training set
second_list = [] # to add to test set
current = [] # contains all 24 sequences from the same DNA section

for i in range(len(seq_record_list)):
    current.append(seq_record_list.pop())
    if len(current) == 24:
        shuffle(current) # Shuffle the 24 sequences from the same DNA section
        random_select = random.randint(18, 24) # Allocate the number of sequences to the training set
        first_list.extend(current[:random_select])
        second_list.extend(current[random_select:])
        current = []

shuffle(first_list) # Shuffle again to eliminate dependencies
shuffle(second_list) # Shuffle again to eliminate dependencies

seq_record_list = first_list + second_list

print("Number of sequences in training/validation set are: " + str(len(first_list)))
print("Number of sequences in testing set are: " + str(len(second_list)))

Number of sequences in training/validation set are: 7114
Number of sequences in testing set are: 974


In [8]:
train_val_num = len(first_list)
test_num = len(second_list)

In [9]:
X_train, y_train, X_test, y_test = prepare_input(train_val_num, test_num, curtail_len, seq_record_list, motif_num)
X_train, y_train, X_test, y_test = to_np_array(X_train, y_train, X_test, y_test)

# Check the shape of training and testing data
[X_train.shape, y_train.shape, X_test.shape, y_test.shape]

[(7114, 21000), (7114, 1), (974, 21000), (974, 1)]

In [10]:
from keras.models import Model, Sequential
from keras.layers import Dense, Conv1D, MaxPooling1D, Dropout, Flatten
from keras.activations import relu
from keras.optimizers import Adam

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [11]:
X_train_cnn = np.expand_dims(X_train, axis=2)
X_train_cnn.shape

(7114, 21000, 1)

In [12]:
LR = 5e-2
model = Sequential()
model.add(Conv1D(filters=1, kernel_size=168, input_shape=(21000, 1), activation="relu", use_bias=True))
model.add(MaxPooling1D())
model.add(Conv1D(filters=2, kernel_size=84, activation="relu", use_bias=True))
model.add(MaxPooling1D())
model.add(Conv1D(filters=4, kernel_size=42, activation="relu", use_bias=True))
model.add(MaxPooling1D())
model.add(Conv1D(filters=8, kernel_size=21, activation="relu", use_bias=True))
model.add(MaxPooling1D())
model.add(Flatten())
model.add(Dense(1000, activation="relu"))
model.add(Dropout(0.7))
model.add(Dense(1, activation="sigmoid"))

In [13]:
model.compile(optimizer=Adam(lr=LR), 
                             loss='binary_crossentropy',
                             metrics=['acc'])
history = model.fit(X_train_cnn, y_train, epochs=30, batch_size=128, validation_split=0.1)

Train on 6402 samples, validate on 712 samples
Epoch 1/30


ResourceExhaustedError: OOM when allocating tensor with shape[1] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[Node: dense_2/bias/Assign = Assign[T=DT_FLOAT, use_locking=true, validate_shape=true, _device="/job:localhost/replica:0/task:0/device:GPU:0"](dense_2/bias, conv1d_1/Const)]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.


Caused by op 'dense_2/bias/Assign', defined at:
  File "/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/ubuntu/anaconda3/envs/tensorflow_p36/li